# Finetune Testing

In [4]:
# load libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import os
import time
import re, string

# Load sklearn packages
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report


#%matplotlib inline

### Google Credentials

In [5]:
if 2==1:    
    # Code to read csv file into colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

## Get the data

In [16]:
#from google.colab import files

# Test Data
testpredictions=pd.read_csv(open('finetune_test_predictions.tsv','rU'), encoding='utf-8', engine='c',sep='\t')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


In [18]:
display(testpredictions.head(2))
display(testpredictions.columns)

,Unnamed: 0,testX,testY,testY_MAIN,predictions,predictions_MAIN
0,88242,mr davis need two bump signs dispatched nnn nn...,street_general,street,street_repair,street
1,1094858,nnn nnth ave,planning_general,planning,environment_dumping,environment


Index(['Unnamed: 0', 'testX', 'testY', 'testY_MAIN', 'predictions',
       'predictions_MAIN'],
      dtype='object')

In [24]:
print('testX NaNs:',testpredictions['testX'].isna().sum())

print('testY NaNs:',testpredictions['testY'].isna().sum())
print('testY_MAIN NaNs:',testpredictions['testY_MAIN'].isna().sum())

print('predictions NaNs:',testpredictions['predictions'].isna().sum())
print('predictions_MAIN NaNs:',testpredictions['predictions_MAIN'].isna().sum())

testX NaNs: 632
testY NaNs: 2109
testY_MAIN NaNs: 2109
predictions NaNs: 2109
predictions_MAIN NaNs: 3586


## Clean the data

- remove rows with NaN in "testX"
- remove rows with NaN in "testY"

In [28]:
testpredictions_cleaned=testpredictions.dropna(axis=0,subset=['testX'])
print('dropped na from testX')
print('testX NaNs:',testpredictions_cleaned['testX'].isna().sum())
print('testY NaNs:',testpredictions_cleaned['testY'].isna().sum())
print('testY_MAIN NaNs:',testpredictions_cleaned['testY_MAIN'].isna().sum())
print('predictions NaNs:',testpredictions_cleaned['predictions'].isna().sum())
print('predictions_MAIN NaNs:',testpredictions_cleaned['predictions_MAIN'].isna().sum())

# now from testY
testpredictions_cleaned=testpredictions_cleaned.dropna(axis=0,subset=['testY'])
print('\ndropped na from testY')
print('testX NaNs:',testpredictions_cleaned['testX'].isna().sum())
print('testY NaNs:',testpredictions_cleaned['testY'].isna().sum())
print('testY_MAIN NaNs:',testpredictions_cleaned['testY_MAIN'].isna().sum())
print('predictions NaNs:',testpredictions_cleaned['predictions'].isna().sum())
print('predictions_MAIN NaNs:',testpredictions_cleaned['predictions_MAIN'].isna().sum())

dropped na from testX
testX NaNs: 0
testY NaNs: 1477
testY_MAIN NaNs: 1477
predictions NaNs: 1477
predictions_MAIN NaNs: 2954
dropped na from testX
testX NaNs: 0
testY NaNs: 0
testY_MAIN NaNs: 0
predictions NaNs: 0
predictions_MAIN NaNs: 1477


In [29]:
testpredictions_cleaned=testpredictions_cleaned.fillna(value='unknown')
print('\nfilled na from predictions_MAIN')
print('testX NaNs:',testpredictions_cleaned['testX'].isna().sum())
print('testY NaNs:',testpredictions_cleaned['testY'].isna().sum())
print('testY_MAIN NaNs:',testpredictions_cleaned['testY_MAIN'].isna().sum())
print('predictions NaNs:',testpredictions_cleaned['predictions'].isna().sum())
print('predictions_MAIN NaNs:',testpredictions_cleaned['predictions_MAIN'].isna().sum())


filled na from predictions_MAIN
testX NaNs: 0
testY NaNs: 0
testY_MAIN NaNs: 0
predictions NaNs: 0
predictions_MAIN NaNs: 0


In [30]:
print('MAIN CLASS')

print(classification_report(testpredictions_cleaned['testY_MAIN'], testpredictions_cleaned['predictions_MAIN']))

MAIN CLASS


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                                precision    recall  f1-score   support

                   environment       0.86      0.97      0.91    113719
    environment_abandoned_site       0.00      0.00      0.00         1
 environment_abandoned_vehicle       0.00      0.00      0.00         5
     environment_air_pollution       0.00      0.00      0.00       175
          environment_asbestos       0.00      0.00      0.00        30
           environment_dumping       0.00      0.00      0.00       103
environment_garbage_collection       0.00      0.00      0.00       424
environment_hazardous_material       0.00      0.00      0.00        31
            environment_litter       0.00      0.00      0.00         4
        environment_overgrowth       0.00      0.00      0.00        42
         environment_recycling       0.00      0.00      0.00       205
                          fire       0.87      0.86      0.86      2359
                    governance       0.58      0.44      0.50  

In [31]:
print('SUB CLASS')

print(classification_report(testpredictions_cleaned['testY'], testpredictions_cleaned['predictions']))

SUB CLASS
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.73      0.15      0.25        54
                    environment       0.96      0.97      0.97      1008
     environment_abandoned_site       0.73      0.57      0.64       828
  environment_abandoned_vehicle       0.81      0.66      0.73      6452
      environment_air_pollution       0.86      0.93      0.89      3387
           environment_asbestos       0.94      0.93      0.94       514
        environment_dead_animal       0.89      0.75      0.81      2311
            environment_dumping       0.62      0.70      0.66      8164
 environment_garbage_collection       0.72      0.97      0.83     54378
            environment_general       0.72      0.53      0.61      1491
 environment_hazardous_material       0.85      0.78      0.81      1111
             environment_litter       0.64      0.56      0.60      3315
         environment_overgrowth       0.